### Heroes Of Pymoli Data Analysis


Top Trends:


* Males make up majority of the player base with 84.03% being male (484/576) Males makes up 82.68% (1,967.64/2,379.77) of the total revenue generated via 652 item purchases. Overall, majority of players are between 15-29 (76.74%) and account for 76.48%   (1,819.95/2,379.77) of the total revenue.



* The average price of all items purchsed is 3.05, but the average price of unique items is 3.04. That means players are more often purchasing items that cost more than the average item price. This trend seems to be due to the spending habits of female and non_disclosed players, spending an average of 3.28 per item between the two groups while males only spent 3.02.



* Oathbreaker, Last Hope of the Breaking Storm is both the most popular and most profitable item earning 50.76 through 12 sales at 4.23 (9 males and 3 females). 4 of the top 5 most popular items are above 3.50 which potentially means players are willing to pay for more expensive items so the price could be increased on some of the less popular items. Looking at purchase habits of the smallest gender demographic, the 11 players in the Other/Non_disclosed gender account for only 15 sales, but 12 of 15 were over 3.05. There were two items purchased more than once. The two most profitable items in that specific player base were Betrayal, Whisper of Grieving Widows and Warped Iron Scimitar totalling 14.08 in sales, 28.05% of the total sales for that gender demographic. 


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

#ignore the red warnings
import warnings
warnings.filterwarnings('ignore')

# File to Load (Remember to Change These)
file_to_load = "../Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df_pd = pd.read_csv(file_to_load)
df_pd.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [2]:
player_count = df_pd["SN"].nunique()
df_player_count = pd.DataFrame({"Total Players":[player_count]})
df_player_count
#print(f"Total number of unique players: {player_count}") 

,Total Players
0,576


## Purchasing Analysis (Total)

*NOTE: The assignment calculates average item price at 3.05, but that includes duplicate items. I have put that calculation as the Average Spent on Items and calculated the average item price using an unduplicated list of items.

In [3]:
#Create calculations and a dataframe to display the total purchases analysis

#Count Unique Items
uniqueItems = df_pd["Item ID"].nunique()

#count number of items sold
items_sold = df_pd["Purchase ID"].count()

#Calculate total revenue by summing price column
totalRev = df_pd["Price"].sum()

#Find Mean to calculate Avg spent on items. 
avgPrchs = round(df_pd["Price"].mean(), 2)

#Eliminate Duplicate Items before calculating average item price.
t_item_analysis = df_pd[['Item Name', 'Price']]
t_item_analysis = t_item_analysis.sort_values('Item Name')
t_item_analysis = t_item_analysis.drop_duplicates()

#average price of items after duplicates removed to get actual item avg price
avgPrice = round(t_item_analysis["Price"].mean(), 2)

#create a dataframe to store the values
purchase_summary = pd.DataFrame({"Unique Items": uniqueItems, "Average Item Price ($)": avgPrice, "Total Items Sold":items_sold
                                 ,"Average Spent on Items ($)":avgPrchs,"Total Revenue ($)":"{:,}".format(totalRev)},index=["Data"])
#output dataframe
purchase_summary

,Unique Items,Average Item Price ($),Total Items Sold,Average Spent on Items ($),Total Revenue ($)
Data,183,3.04,780,3.05,"2,379.77"


## Gender Demographics

In [4]:
#Create a data frame to summarize the player gender demographics

#Create an unduplicated list of players
df_players = df_pd[['SN', 'Age','Gender']]
df_players = df_players.drop_duplicates()

#Calculate the number of of players for each gender
male_players = df_players.loc[df_players["Gender"] == "Male","Gender"].count()
female_players = df_players.loc[df_players["Gender"] == "Female", "Gender"].count()
nd_players = df_players.loc[df_players["Gender"] == "Other / Non-Disclosed", "Gender"].count()

#calculate the percentage of player for each gender 
male_percent = round(male_players/player_count, 4) 
female_percent = round(female_players/player_count, 4) 
nd_percent = round(nd_players/player_count, 4) 


#Create a dataframe to store the calculated values
gender_demo = pd.DataFrame({"Total Players": [player_count], "Male Players":[male_players], "Male Percent":"{:.2%}".format(male_percent)
                            , "Female Players":[female_players], "Female Percent":"{:.2%}".format(female_percent)
                            , "Non-Disclosed":[nd_players], "Non-Disclosed Percent":"{:.2%}".format(nd_percent)})

#Output the Gender Demographics
gender_demo.head()

,Total Players,Male Players,Male Percent,Female Players,Female Percent,Non-Disclosed,Non-Disclosed Percent
0,576,484,84.03%,81,14.06%,11,1.91%



## Purchasing Analysis (Gender)

In [5]:
#create data frame for non-duplicated player counts
gndr_players = df_pd[['SN','Gender']]
gndr_players = gndr_players.drop_duplicates()

#group the distinct user dataframe by gender and rename the column
gndr_join = pd.DataFrame(gndr_players.groupby(["Gender"]).count())
gndr_join = gndr_join.rename(columns = {"SN":"Unique Players"})

#Create data frame for gender/price calcualtions to join with non-dup counts
gndr_sales2 = pd.DataFrame(df_pd[['Gender','Price']])
gndr_sales3 = gndr_sales2.rename(columns = {"Price":"Purchase Count"})

#calculations for gender analysis
gndr_count = pd.DataFrame(gndr_sales3.groupby(["Gender"]).count())
gndr_count["Total Purchase Value ($)"]= gndr_sales3.groupby(["Gender"]).sum()
gndr_count["Average Purchase Price ($)"] = round(gndr_sales3.groupby(["Gender"]).mean(), 2)

#calculate percent of total sales and percent of total revenue
gndr_count["% of Total Sales"] = round(gndr_count["Purchase Count"] / len(gndr_sales2), 4) 
gndr_count["% of Total Rev"] = round(gndr_count["Total Purchase Value ($)"] / totalRev, 4) 

gndr_count["% of Total Sales"] = gndr_count["% of Total Sales"].map("{:.2%}".format)
gndr_count["% of Total Rev"] = gndr_count["% of Total Rev"].map("{:.2%}".format)

#join the data frames for the last calculation
gndr_join_sum = gndr_join.join(gndr_count)
gndr_join_sum["Avg Total Purchase per Person ($)"] = round(gndr_join_sum["Total Purchase Value ($)"] / gndr_join_sum["Unique Players"], 2)

#output the summary dataframe
gndr_join_sum


,Unique Players,Purchase Count,Total Purchase Value ($),Average Purchase Price ($),% of Total Sales,% of Total Rev,Avg Total Purchase per Person ($)
Gender,,,,,,,
Female,81,113,361.94,3.20,14.49%,15.21%,4.47
Male,484,652,1967.64,3.02,83.59%,82.68%,4.07
Other / Non-Disclosed,11,15,50.19,3.35,1.92%,2.11%,4.56


## Age Demographics

In [6]:
#Create bins and labels to summarize the age demographics and put the data into a dataframe

#Create bins to separate the age groups
bins = [0, 9, 14, 19,24,29,34,39,200]

#create age groups to label the binned data
age_groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#create an unduplicated dataset
ages = df_pd[["SN","Age"]]
ages = ages.drop_duplicates(["SN"])

#cut the data into the labeled bins
ages["Age Groups"] = pd.cut(ages["Age"], bins, labels = age_groups)
ages = ages[["Age Groups","Age"]]            

#Calculate the percentage of players
age_group = ages.groupby(["Age Groups"]).count()
age_group = age_group.rename(columns = {"Age":"Total Count"})
age_group["Percentage of Players"] = round((age_group["Total Count"] / player_count), 4)
age_group["Percentage of Players"] = age_group["Percentage of Players"].map("{:.2%}".format)

#display dataframe
age_group

,Total Count,Percentage of Players
Age Groups,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [7]:
#create age groups and bin the data to summarize purchases by age groups


bins2 = [0, 9, 14, 19, 24, 29, 34, 39, 200]
age_groups2 = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#create a dataframe that includes duplicates to analyze the purchase data
ages_prchs = pd.DataFrame(df_pd[["SN","Age", "Price"]])

#cut the data to into the age groups
ages_prchs["Age Groups"] = pd.cut(ages_prchs["Age"], bins2, labels = age_groups2)

#count the data for purchase counts
ages_prchs_g = ages_prchs.groupby(["Age Groups"]).count()
ages_prchs_g = ages_prchs_g.rename(columns = {"Age":"Purchase Count"})

#sum the data for total purchase value
ages_prchs_s = round(ages_prchs.groupby(["Age Groups"]).sum(), 2)
ages_prchs_s = ages_prchs_s.rename(columns = {"Price":"Total Purchase Value ($)"})

#calculate the mean to get the average purchase price
ages_prchs_m = round(ages_prchs.groupby(["Age Groups"]).mean(), 2)
ages_prchs_m = ages_prchs_m.rename(columns = {"Price":"Average Purchase Price ($)"})

#Merge the datasets together
merge1_ap = ages_prchs_g.merge(ages_prchs_s, how = "inner", on = "Age Groups")
merge2_ap = merge1_ap.merge(ages_prchs_m, how = "inner", on = "Age Groups")

#select the column for the summary dataframe
age_prchs_sum = merge2_ap[["Purchase Count", "Total Purchase Value ($)", "Average Purchase Price ($)"]]

#add the last column for caluclating avg total purchase per person
age_prchs_sum["Avg Total Purchase per Person ($)"] = round(age_prchs_sum["Total Purchase Value ($)"] / age_group["Total Count"], 2)

#format the data with dollar signs
# age_prchs_sum["Total Purchase Value"] = age_prchs_sum["Total Purchase Value"].map("${:}".format)
# age_prchs_sum["Avg Total Purchase per Person"] = age_prchs_sum["Avg Total Purchase per Person"].map("${:.3}".format)
# age_prchs_sum["Average Purchase Price"] = age_prchs_sum["Average Purchase Price"].map("${:.3}".format)

age_prchs_sum

,Purchase Count,Total Purchase Value ($),Average Purchase Price ($),Avg Total Purchase per Person ($)
Age Groups,,,,
<10,23,77.13,3.35,4.54
10-14,28,82.78,2.96,3.76
15-19,136,412.89,3.04,3.86
20-24,365,1114.06,3.05,4.32
25-29,101,293.00,2.90,3.81
30-34,73,214.00,2.93,4.12
35-39,41,147.67,3.60,4.76
40+,13,38.24,2.94,3.19


## Top Spenders

In [8]:
#Create data frames to calculate the purchase data per user and merge them to form a summary data frame.

#select columns to calculate top spenders
top_spenders = pd.DataFrame(df_pd[["SN","Item ID", "Price"]])

#Count the data for puchrase counts
top_spenders_g = top_spenders.groupby("SN").count()
top_spenders_g = top_spenders_g.rename(columns = {"Item ID":"Purchase Count"})

#take the mean for average pruchase price
top_spenders_m = round(top_spenders.groupby("SN").mean(),2)
top_spenders_m = top_spenders_m.rename(columns = {"Price":"Average Purchase Price ($)"})

#Sum the data for total purchase value
top_spenders_t = top_spenders.groupby("SN").sum()
top_spenders_t = top_spenders_t.rename(columns = {"Price":"Total Purchase Value ($)"})

#merge the dataframes into the summary frame
merge1_ts = top_spenders_g.merge(top_spenders_m, how = "inner", on = "SN")
merge2_ts = merge1_ts.merge(top_spenders_t, how = "inner", on = "SN")

#select the columns to display in the summary table
age_prchs_sum = merge2_ts[["Purchase Count", "Average Purchase Price ($)", "Total Purchase Value ($)"]]

#sort the values to identify the top spender by user
age_prchs_sum = age_prchs_sum.sort_values("Total Purchase Value ($)", ascending = False)

#format the columns to add dollar signs
# age_prchs_sum["Average Purchase Price"] = age_prchs_sum["Average Purchase Price"].map("${:.2f}".format)
# age_prchs_sum["Total Purchase Value"] = age_prchs_sum["Total Purchase Value"].map("${:.2f}".format)

#output the dataframe
age_prchs_sum.head()

,Purchase Count,Average Purchase Price ($),Total Purchase Value ($)
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


## Most Popular Items

In [9]:
#create a dataframe to calculate the most popular items

#select the columns to include in the dataframe
popular_items = df_pd[["Item ID","Item Name", "Price"]]

#rename the column
popular_items = popular_items.rename(columns = {"Price":"Purchase Count"})

#Use group by to create a multi-index dataframe and count the data in the renamed column
popular_items_g = popular_items.groupby(["Item ID", "Item Name"]).count()

#Calculate the item price and total purchase value
popular_items_g["Item Price ($)"] = popular_items.groupby(["Item ID", "Item Name"]).mean()

#format the column to add a dollar sign
#popular_items_g["Item Price"] = popular_items_g["Item Price"] #.map("${:.2f}".format)
popular_items_g["Total Purchase Value ($)"] = popular_items.groupby(["Item ID", "Item Name"]).sum()

#sort the values by purchase count
popular_items_gs = popular_items_g.sort_values("Purchase Count", ascending = False)

#format the column with a $
#popular_items_gs["Total Purchase Value"] = popular_items_gs["Total Purchase Value"] #.map("${:.2f}".format)

#display the dataframe
popular_items_gs.head()

,,Purchase Count,Item Price ($),Total Purchase Value ($)
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


## Most Profitable Items

In [10]:
#use the previous dataframe to find the most profitable item

profitable_items_g = popular_items_g.sort_values("Total Purchase Value ($)", ascending = False)
#format the column with a $ sign
#profitable_items_g["Total Purchase Value"] = popular_items_g["Total Purchase Value"].map("${:.2f}".format)

profitable_items_g.head()

,,Purchase Count,Item Price ($),Total Purchase Value ($)
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80


In [11]:
#Used for querying the data to identify additional trend data

# popular_items2 = df_pd[["Item ID","Item Name", "Price", "Gender", "SN"]]
# item = "Oathbreaker, Last Hope of the Breaking Storm"
# gender = "Other / Non-Disclosed"
# popular_items2 = popular_items2.loc[popular_items2["Gender"]== gender,:]

# popular_items2.reset_index().sort_values("Item ID")
